# Scraping web data and text mining for analysis, and for use with a Tensorflow RNN example

In [5]:
from HTMLParser import HTMLParser

In [6]:
parser = HTMLParser()

In [8]:
parser.feed('https://docs.python.org/2/library/htmlparser.html')